In [8]:
import os
import sys

# If you like to speed up training process with GPU, first install PlaidML and then uncomment the following line.
# Otherwise it will fallback to tensorflow.

# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


In [9]:
print(sys.argv)
print(len(sys.argv))

['D:\\anaconda3\\envs\\tensorflow\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Ashutosh Shukla\\AppData\\Roaming\\jupyter\\runtime\\kernel-8da6eb5a-8ef6-4278-a898-0fbf0cdf70cf.json']
3


In [10]:
# !pip install tensorflow --upgrade
# !pip install intel-tensorflow --user
# !pip install numpy==1.21.6
# !pip uninstall tensorflow
# !pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense, Dropout, MaxPool2D, Conv2D

from keras import optimizers
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [11]:

print(tf.__version__)

2.9.0


In [12]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
    DEV = True

if DEV:
    epochs = 10
else:
    epochs = 1000

# BEFORE STARTING TRAINING YOU NEED TO MANUALLY TAKE 20 PERCENENT OF THE TRAINING DATA AND PUT IT INTO VALIDATION FOLDER
# I was too lazy to do it in the code.

train_data_dir = 'D:/All files/CryptoData/ImageNN/Data_split_V2/train'
validation_data_dir = 'D:/All files/CryptoData/ImageNN/Data_split_V2/val'

In [14]:
# Input the size of your sample images
img_width, img_height = 256, 256 #150, 150
# Enter the number of samples, training + validation
nb_train_samples = 325671
nb_validation_samples = 93071
# nb_filters1 = 32
# nb_filters2 = 32
# nb_filters3 = 64
# conv1_size = 3
# conv2_size = 2
# conv3_size = 5
# pool_size = 2
# We have 2 classes, buy and sell
# classes_num = 2
batch_size = 128 #128
# lr = 0.01 #0.001
# chanDim =3

In [19]:
model = keras.Sequential([
    
    keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding="same", input_shape=(256, 256, 3), kernel_regularizer='l2'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), data_format='channels_last'),
    
    keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding="same", kernel_regularizer='l2'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.MaxPool2D(pool_size=(2,2), data_format='channels_last'),
    
    keras.layers.Conv2D(filters=64, kernel_size=2, strides=1, padding="same", kernel_regularizer='l2'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.MaxPool2D(pool_size=(2,2), data_format='channels_last'),
    
#     keras.layers.Conv2D(filters=1024, kernel_size=3, strides=2, padding="same", kernel_regularizer='l2'),
#     keras.layers.BatchNormalization(),
#     keras.layers.Activation('elu'),
#     keras.layers.MaxPool2D(pool_size=(2, 2), data_format='channels_last'),
    
#     keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding="same", kernel_regularizer='l2'),
#     keras.layers.BatchNormalization(),
#     keras.layers.Activation('elu'),
#     keras.layers.MaxPool2D(pool_size=(2, 2), data_format='channels_last'),

    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dense(128),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dropout(0.6),
    
    keras.layers.Dense(2),
    keras.layers.Softmax()
    ])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 128)     3584      
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 128)    512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 128, 128, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 128)      5

In [20]:
# Use earlystopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, min_delta=0.01, restore_best_weights=True)
opt = tf.keras.optimizers.Nadam() #learning_rate=0.001

# fitting the model
model.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])
#                       callbacks=[callback])
# epoch = 0
# val_acc = 0

In [15]:
# train_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     horizontal_flip=False)

# test_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     horizontal_flip=False)

# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_height, img_width),
#     shuffle=True,
#     batch_size=batch_size,
#     class_mode='categorical')

# validation_generator = test_datagen.flow_from_directory(
#     validation_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     shuffle=True,
#     class_mode='categorical')

Found 325611 images belonging to 2 classes.
Found 93031 images belonging to 2 classes.


In [ ]:
"""
Tensorboard log
"""
target_dir = f"D:/All files/CryptoData/ImageNN/models/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
if not os.path.exists(target_dir):
    os.mkdir(target_dir)
model.save('D:/All files/CryptoData/ImageNN/models/model.h5')
model.save_weights('D:/All files/CryptoData/ImageNN/models/weights.h5')

In [20]:
# checkpoint = ModelCheckpoint(target_dir, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [tf.keras.callbacks.TensorBoard(log_dir='D:/All files/CryptoData/ImageNN/models/logs/', histogram_freq=1)]  #[checkpoint]

In [ ]:
# model = load_model('D:/All files/CryptoData/ImageNN/models/model.h5')
model.fit(
    train_generator,
    batch_size=batch_size,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs= 500,
    shuffle=True,
    validation_data=validation_generator,
    callbacks= callbacks_list,
    validation_steps=nb_validation_samples//batch_size)

Epoch 1/500
2544/2544 [==============================] - 1150s 450ms/step - loss: 0.8123 - accuracy: 0.5102 - val_loss: 0.6962 - val_accuracy: 0.5214
Epoch 2/500
2544/2544 [==============================] - 385s 151ms/step - loss: 0.6998 - accuracy: 0.5115 - val_loss: 0.7011 - val_accuracy: 0.4785
Epoch 3/500
2544/2544 [==============================] - 943s 371ms/step - loss: 0.7017 - accuracy: 0.5133 - val_loss: 0.6980 - val_accuracy: 0.5214
Epoch 4/500
2544/2544 [==============================] - 498s 196ms/step - loss: 0.7015 - accuracy: 0.5133 - val_loss: 0.6993 - val_accuracy: 0.5189
Epoch 5/500
2544/2544 [==============================] - 388s 153ms/step - loss: 0.7007 - accuracy: 0.5131 - val_loss: 0.7046 - val_accuracy: 0.4917
Epoch 6/500
2544/2544 [==============================] - 389s 153ms/step - loss: 0.7003 - accuracy: 0.5135 - val_loss: 0.7044 - val_accuracy: 0.5214
Epoch 7/500
2544/2544 [==============================] - 390s 153ms/step - loss: 0.6993 - accuracy: 0.516

In [1]:
import os
import re
import sys
import time
import numpy as np
from typing import Any, List, Tuple, Union
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPool2D, Conv2D, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
import pickle
from keras.preprocessing.image import ImageDataGenerator





def generate_output_dir(outdir, run_desc):
    prev_run_dirs = []
    if os.path.isdir(outdir):
        prev_run_dirs = [x for x in os.listdir(outdir) if os.path.isdir(os.path.join(outdir, x))]
    prev_run_ids = [re.match(r'^\d+', x) for x in prev_run_dirs]
    prev_run_ids = [int(x.group()) for x in prev_run_ids if x is not None]
    cur_run_id = max(prev_run_ids, default=-1) + 1
    run_dir = os.path.join(outdir, f'{cur_run_id:05d}-{run_desc}')
    assert not os.path.exists(run_dir)
    os.makedirs(run_dir)
    return run_dir

# From StyleGAN2
class Logger(object):
    """Redirect stderr to stdout, optionally print stdout to a file, and 
    optionally force flushing on both stdout and the file."""

    def __init__(self, file_name: str = None, file_mode: str = "w", should_flush: bool = True):
        self.file = None

        if file_name is not None:
            self.file = open(file_name, file_mode)

        self.should_flush = should_flush
        self.stdout = sys.stdout
        self.stderr = sys.stderr

        sys.stdout = self
        sys.stderr = self

    def __enter__(self) -> "Logger":
        return self

    def __exit__(self, exc_type: Any, exc_value: Any, traceback: Any) -> None:
        self.close()

    def write(self, text: str) -> None:
        """Write text to stdout (and a file) and optionally flush."""
        if len(text) == 0: 
            return

        if self.file is not None:
            self.file.write(text)

        self.stdout.write(text)

        if self.should_flush:
            self.flush()

    def flush(self) -> None:
        """Flush written text to both stdout and a file, if open."""
        if self.file is not None:
            self.file.flush()

        self.stdout.flush()

    def close(self) -> None:
        """Flush, close possible files, and remove 
            stdout/stderr mirroring."""
        self.flush()

        # if using multiple loggers, prevent closing in wrong order
        if sys.stdout is self:
            sys.stdout = self.stdout
        if sys.stderr is self:
            sys.stderr = self.stderr

        if self.file is not None:
            self.file.close()

def obtain_data():
#     (x_train, y_train), (x_test, y_test) = mnist.load_data()
#     print("Shape of x_train: {}".format(x_train.shape))
#     print("Shape of y_train: {}".format(y_train.shape))
#     print()
#     print("Shape of x_test: {}".format(x_test.shape))
#     print("Shape of y_test: {}".format(y_test.shape))

#     # input image dimensions
#     img_rows, img_cols = 28, 28
#     if K.image_data_format() == 'channels_first':
#         x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#         x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#         input_shape = (1, img_rows, img_cols)
#     else:
#         x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#         x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#         input_shape = (img_rows, img_cols, 1)
#     x_train = x_train.astype('float32')
#     x_test = x_test.astype('float32')
#     x_train /= 255
#     x_test /= 255
#     print('x_train shape:', x_train.shape)
#     print("Training samples: {}".format(x_train.shape[0]))
#     print("Test samples: {}".format(x_test.shape[0]))
#     # convert class vectors to binary class matrices
#     y_train = tf.keras.utils.to_categorical(y_train, num_classes)
#     y_test = tf.keras.utils.to_categorical(y_test, num_classes)
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        horizontal_flip=False)

    test_datagen = ImageDataGenerator(
        rescale=1. / 255,
        horizontal_flip=False)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        shuffle=True,
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')
    
    return train_datagen, test_datagen, train_generator, validation_generator

In [2]:
outdir = "D:/All files/CryptoData/ImageNN/models/"
run_desc = "test-train"
batch_size = 128
nb_train_samples = 325671
nb_validation_samples = 93071

In [3]:
run_dir = generate_output_dir(outdir, run_desc)
print(f"Results saved to: {run_dir}")

Results saved to: D:/All files/CryptoData/ImageNN/models/00015-test-train


In [4]:
class MyModelCheckpoint(ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch,logs)\

        # Also save the optimizer state
        filepath = self._get_file_path(epoch=epoch, logs=logs, batch=None)
        filepath = filepath.rsplit( ".", 1 )[ 0 ] 
        filepath += ".pkl"

        with open(filepath, 'wb') as fp:
            pickle.dump(
                {
                  'opt': model.optimizer.get_config(),
                  'epoch': epoch+1
                 # Add additional keys if you need to store more values
                }, fp, protocol=pickle.HIGHEST_PROTOCOL)
        print('\nEpoch %05d: saving optimizer to %s' % (epoch + 1, filepath))

In [5]:
# def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
#     def schedule(epoch):
#         return initial_lr * (decay_factor ** np.floor(epoch/step_size))
#     return LearningRateScheduler(schedule)

In [6]:
def build_model():
    model = keras.Sequential([

        keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding="same", input_shape=(256, 256, 3), kernel_regularizer='l2'),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('elu'),
        keras.layers.MaxPool2D(pool_size=(2, 2), data_format='channels_last'),

        keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding="same", kernel_regularizer='l2'),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('elu'),
        keras.layers.MaxPool2D(pool_size=(2,2), data_format='channels_last'),

        keras.layers.Conv2D(filters=64, kernel_size=2, strides=1, padding="same", kernel_regularizer='l2'),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('elu'),
        keras.layers.MaxPool2D(pool_size=(2,2), data_format='channels_last'),

    #     keras.layers.Conv2D(filters=1024, kernel_size=3, strides=2, padding="same", kernel_regularizer='l2'),
    #     keras.layers.BatchNormalization(),
    #     keras.layers.Activation('elu'),
    #     keras.layers.MaxPool2D(pool_size=(2, 2), data_format='channels_last'),

    #     keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding="same", kernel_regularizer='l2'),
    #     keras.layers.BatchNormalization(),
    #     keras.layers.Activation('elu'),
    #     keras.layers.MaxPool2D(pool_size=(2, 2), data_format='channels_last'),

        keras.layers.Flatten(),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('elu'),
        keras.layers.Dense(128),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('elu'),
        keras.layers.Dropout(0.6),

        keras.layers.Dense(2),
        keras.layers.Softmax()
        ])
    return model

In [7]:
def train_model(model, initial_epoch=0, max_epochs=500):
    start_time = time.time()

    checkpoint_cb = MyModelCheckpoint(
        os.path.join(run_dir, 'model-{epoch:02d}-{val_loss:.2f}.hdf5'),
        monitor='val_loss',verbose=1)

#     lr_sched_cb = step_decay_schedule(initial_lr=1e-4, decay_factor=0.75, step_size=2)
    cb = [checkpoint_cb] #, lr_sched_cb
    model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Nadam(),
                      metrics=['accuracy'])

    model.fit(
#         x_train, y_train,
#         batch_size=batch_size,
#         epochs=max_epochs,
#         initial_epoch = initial_epoch,
#         verbose=2, callbacks=cb,
#         validation_data=(x_test, y_test)
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs= max_epochs,
        initial_epoch = initial_epoch,
        shuffle=True,
        validation_data=validation_generator,
        callbacks= cb,
        validation_steps=nb_validation_samples//batch_size)
    
#     score = model.evaluate(x_test, y_test, verbose=0, callbacks=cb)
#     print('Test loss: {}'.format(score[0]))
#     print('Test accuracy: {}'.format(score[1]))

    elapsed_time = time.time() - start_time
    print("Elapsed time: {}".format(hms_string(elapsed_time)))

In [9]:
with Logger(os.path.join(run_dir, 'log.txt')):
    train_data_dir = 'D:/All files/CryptoData/ImageNN/Data_split_V2/train'
    validation_data_dir = 'D:/All files/CryptoData/ImageNN/Data_split_V2/val'
    img_height, img_width = 256, 256
    train_datagen, test_datagen, train_generator, validation_generator = obtain_data()
    model = build_model()
    train_model(model, max_epochs=500)

Found 325611 images belonging to 2 classes.
Found 93031 images belonging to 2 classes.
Epoch 1/500
  26/2544 [..............................] - ETA: 15:20 - loss: 3.2302 - accuracy: 0.5276

KeyboardInterrupt: 

In [1]:
!ls D:/All files/CryptoData/ImageNN/models

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!ls ./data/00000-test-train

'ls' is not recognized as an internal or external command,
operable program or batch file.


# For Continuing Training if stopped 

In [8]:
MODEL_PATH = 'D:/All files/CryptoData/ImageNN/models/00014-cont-train/model-1330-0.06.hdf5'
OPT_PATH = 'D:/All files/CryptoData/ImageNN/models/00014-cont-train/model-1330-0.06.pkl'

In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

def load_model_data(model_path, opt_path):
    model = load_model(model_path)
    with open(opt_path, 'rb') as fp:
      d = pickle.load(fp)
      epoch = d['epoch']
      opt = d['opt']
      return epoch, model, opt

epoch, model, opt = load_model_data(MODEL_PATH, OPT_PATH)

# note: often it is not necessary to recompile the model
model.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam.from_config(opt),
    metrics=['accuracy'])
model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Nadam(opt),
                      metrics=['accuracy'])

In [ ]:
outdir = "D:/All files/CryptoData/ImageNN/models/"
run_desc = "cont-train"


run_dir = generate_output_dir(outdir, run_desc)
print(f"Results saved to: {run_dir}")

with Logger(os.path.join(run_dir, 'log.txt')):
    train_data_dir = 'D:/All files/CryptoData/ImageNN/Data_split_V2/train'
    validation_data_dir = 'D:/All files/CryptoData/ImageNN/Data_split_V2/val'
    img_height, img_width = 256, 256
    train_datagen, test_datagen, train_generator, validation_generator = obtain_data()
    train_model(model, initial_epoch=epoch, max_epochs=5000)

Results saved to: D:/All files/CryptoData/ImageNN/models/00016-cont-train
Found 325611 images belonging to 2 classes.
Found 93031 images belonging to 2 classes.
Epoch 1331/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0637 - accuracy: 0.9866
Epoch 1331: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1331-0.09.hdf5

Epoch 01331: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1331-0.09.pkl
2544/2544 [==============================] - 1379s 538ms/step - loss: 0.0637 - accuracy: 0.9866 - val_loss: 0.0880 - val_accuracy: 0.9835
Epoch 1332/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0624 - accuracy: 0.9871
Epoch 1332: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1332-0.08.hdf5

Epoch 01332: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1332-0.08.pkl
2544/2544 [==============================] - 400s 157ms/step - lo

Epoch 01366: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1366-2.52.pkl
2544/2544 [==============================] - 411s 161ms/step - loss: 0.0644 - accuracy: 0.9865 - val_loss: 2.5160 - val_accuracy: 0.7599
Epoch 1367/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0647 - accuracy: 0.9868
Epoch 1367: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1367-1.92.hdf5

Epoch 01367: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1367-1.92.pkl
2544/2544 [==============================] - 412s 162ms/step - loss: 0.0647 - accuracy: 0.9868 - val_loss: 1.9178 - val_accuracy: 0.7958
Epoch 1368/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0637 - accuracy: 0.9868
Epoch 1368: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1368-0.11.hdf5

Epoch 01368: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\m

Epoch 01402: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1402-1.08.pkl
2544/2544 [==============================] - 410s 161ms/step - loss: 0.0631 - accuracy: 0.9868 - val_loss: 1.0787 - val_accuracy: 0.8351
Epoch 1403/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0637 - accuracy: 0.9867
Epoch 1403: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1403-0.32.hdf5

Epoch 01403: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1403-0.32.pkl
2544/2544 [==============================] - 414s 163ms/step - loss: 0.0637 - accuracy: 0.9867 - val_loss: 0.3190 - val_accuracy: 0.9521
Epoch 1404/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0643 - accuracy: 0.9865
Epoch 1404: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1404-0.06.hdf5

Epoch 01404: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\m

Epoch 01438: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1438-0.07.pkl
2544/2544 [==============================] - 821s 323ms/step - loss: 0.0626 - accuracy: 0.9871 - val_loss: 0.0651 - val_accuracy: 0.9910
Epoch 1439/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0637 - accuracy: 0.9869
Epoch 1439: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1439-0.06.hdf5

Epoch 01439: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1439-0.06.pkl
2544/2544 [==============================] - 823s 323ms/step - loss: 0.0637 - accuracy: 0.9869 - val_loss: 0.0579 - val_accuracy: 0.9913
Epoch 1440/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0633 - accuracy: 0.9871
Epoch 1440: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1440-0.06.hdf5

Epoch 01440: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\m

Epoch 01474: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1474-0.06.pkl
2544/2544 [==============================] - 413s 162ms/step - loss: 0.0632 - accuracy: 0.9871 - val_loss: 0.0569 - val_accuracy: 0.9924
Epoch 1475/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0630 - accuracy: 0.9870
Epoch 1475: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1475-1.67.hdf5

Epoch 01475: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1475-1.67.pkl
2544/2544 [==============================] - 403s 158ms/step - loss: 0.0630 - accuracy: 0.9870 - val_loss: 1.6716 - val_accuracy: 0.7820
Epoch 1476/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0631 - accuracy: 0.9871
Epoch 1476: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1476-0.06.hdf5

Epoch 01476: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\m

Epoch 01510: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1510-0.07.pkl
2544/2544 [==============================] - 385s 151ms/step - loss: 0.0631 - accuracy: 0.9871 - val_loss: 0.0682 - val_accuracy: 0.9915
Epoch 1511/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0631 - accuracy: 0.9869
Epoch 1511: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1511-0.13.hdf5

Epoch 01511: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1511-0.13.pkl
2544/2544 [==============================] - 384s 151ms/step - loss: 0.0631 - accuracy: 0.9869 - val_loss: 0.1324 - val_accuracy: 0.9698
Epoch 1512/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0633 - accuracy: 0.9869
Epoch 1512: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1512-0.06.hdf5

Epoch 01512: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\m

Epoch 01546: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1546-0.13.pkl
2544/2544 [==============================] - 402s 158ms/step - loss: 0.0634 - accuracy: 0.9869 - val_loss: 0.1281 - val_accuracy: 0.9784
Epoch 1547/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0621 - accuracy: 0.9872
Epoch 1547: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1547-0.06.hdf5

Epoch 01547: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1547-0.06.pkl
2544/2544 [==============================] - 403s 158ms/step - loss: 0.0621 - accuracy: 0.9872 - val_loss: 0.0571 - val_accuracy: 0.9921
Epoch 1548/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0625 - accuracy: 0.9873
Epoch 1548: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1548-1.91.hdf5

Epoch 01548: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\m

Epoch 01582: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1582-0.14.pkl
2544/2544 [==============================] - 404s 159ms/step - loss: 0.0632 - accuracy: 0.9870 - val_loss: 0.1358 - val_accuracy: 0.9761
Epoch 1583/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0617 - accuracy: 0.9874
Epoch 1583: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1583-0.12.hdf5

Epoch 01583: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1583-0.12.pkl
2544/2544 [==============================] - 405s 159ms/step - loss: 0.0617 - accuracy: 0.9874 - val_loss: 0.1176 - val_accuracy: 0.9716
Epoch 1584/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0630 - accuracy: 0.9871
Epoch 1584: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1584-0.09.hdf5

Epoch 01584: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\m

Epoch 01618: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1618-0.08.pkl
2544/2544 [==============================] - 404s 159ms/step - loss: 0.0626 - accuracy: 0.9873 - val_loss: 0.0842 - val_accuracy: 0.9814
Epoch 1619/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0616 - accuracy: 0.9876
Epoch 1619: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1619-0.07.hdf5

Epoch 01619: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1619-0.07.pkl
2544/2544 [==============================] - 404s 159ms/step - loss: 0.0616 - accuracy: 0.9876 - val_loss: 0.0688 - val_accuracy: 0.9889
Epoch 1620/5000
2544/2544 [==============================] - ETA: 0s - loss: 0.0624 - accuracy: 0.9872
Epoch 1620: saving model to D:/All files/CryptoData/ImageNN/models/00016-cont-train\model-1620-2.37.hdf5

Epoch 01620: saving optimizer to D:/All files/CryptoData/ImageNN/models/00016-cont-train\m

# For testing the model with test data

In [ ]:
import os
import numpy as np
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model


img_width, img_height = 256, 256
model_path = 'D:/All files/CryptoData/ImageNN/models/model.h5'
# weights_path = 'D:/All files/CryptoData/ImageNN/models/weights.h5'
model = load_model(model_path)
test_path = 'D:/All files/CryptoData/ImageNN/Data_split_V2/test'

def predict(file):
    x = load_img(file, target_size=(img_width,img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    result = array[0]
    if result[0] > result[1]:
        if result[0] > 0.9:
            print("Predicted answer: Buy")
            answer = 'buy'
            print(result)
            print(array)
        else:
            print("Predicted answer: Not confident")
            answer = 'n/a'
            print(result)
    else:
        if result[1] > 0.9:
            print("Predicted answer: Sell")
            answer = 'sell'
            print(result)
        else:
            print("Predicted answer: Not confident")
            answer = 'n/a'
            print(result)

    return answer


tb = 0
ts = 0
fb = 0
fs = 0
na = 0

for i, ret in enumerate(os.walk(test_path + '/buy')):
    for i, filename in enumerate(ret[2]):
        if filename.startswith("."):
            continue
        print("Label: buy")
        result = predict(ret[0] + '/' + filename)
        if result == "buy":
            tb += 1
        elif result == 'n/a':
            print('no action')
            na += 1
        else:
            fb += 1

for i, ret in enumerate(os.walk(test_path + '/sell')):
    for i, filename in enumerate(ret[2]):
        if filename.startswith("."):
            continue
        print("Label: sell")
        result = predict(ret[0] + '/' + filename)
        if result == "sell":
            ts += 1
        elif result == 'n/a':
            print('no action')
            na += 1
        else:
            fs += 1

"""
Check metrics
"""
print("True buy: ", tb)
print("True sell: ", ts)
print("False buy: ", fb)  # important
print("False sell: ", fs)
print("No action", na)

precision = (tb+ts) / (tb + ts + fb + fs)
recall = tb / (tb + fs)
print("Precision: ", precision)
print("Recall: ", recall)

f_measure = (2 * recall * precision) / (recall + precision)
print("F-measure: ", f_measure)


In [ ]:
# !pip install keras-tuner --upgrade
# import keras_tuner as kt


In [ ]:
# !tensorboard --logdir logs/fit

In [ ]:
# import skimage.io as io
# import matplotlib.pyplot as plt
# import numpy as np

# # Read the image
# img = io.imread('C:\\Users\DELL\Desktop\CryptoData\ImageNN\candle_images\\1st.jpg')
# print(img.shape)
# plt.imshow(img)
# plt.show()

In [ ]:
# import skimage.io as io
# import matplotlib.pyplot as plt
# import numpy as np

# # Read the image
# img = io.imread('C:\\Users\DELL\Desktop\CryptoData\ImageNN\candle_images\sell\\665af8e9-6401-4533-baec-7ca0e9953207.png')
# print(img.shape)
# plt.imshow(img)
# plt.show()

In [ ]:
# import skimage.io as io
# import matplotlib.pyplot as plt
# import numpy as np

# # Read the image
# img = io.imread('C:\\Users\DELL\Desktop\CryptoData\ImageNN\candle_images\sell\\6ea72d3e-60ad-450b-aaad-7ef84d122e5e.png')
# print(img.shape)
# plt.imshow(img)
# plt.show()

In [ ]:
# import skimage.io as io
# import matplotlib.pyplot as plt
# import numpy as np

# # Read the image
# img = io.imread('C:\\Users\DELL\Desktop\CryptoData\ImageNN\candle_images\sell\\9e93410d-1dbe-4b64-9884-a37d0b2fd178.jpg')
# print(img.shape)
# plt.imshow(img)
# plt.show()